# The demonstration of the tropical precipitation module on the different datasets

###  Loading the necessary packages

In [1]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib

import inspect
import timeit
import sys

### Function, which reads the status of the user in the slurm queue 

In [ ]:
#!pip install aqua

In [2]:
def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

### Storing the path to the current repository into the variable

In [3]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

### Setting the slurm job

In [4]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]

cluster = SLURMCluster(
    name='dask-cluster', 
    cores=16,    
    memory="100 GB", 
    project="bb1153",
    queue= "compute", 
    walltime='04:30:50',
    job_extra=extra_args,
)
client = Client(cluster)
print(cluster.job_script())

/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=16
#SBATCH --mem=94G
#SBATCH -t 04:30:50
#SBATCH --error=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/logs/dask-worker-%j.err
#SBATCH --output=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/output/dask-worker-%j.out

/home/b/b382267/mambaforge/envs/tropical-rainfall/bin/python -m distributed.cli.dask_worker tcp://136.172.124.7:41315 --nthreads 4 --nworkers 4 --memory-limit 23.28GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


### Loading the slurm job to the queue 

In [5]:
cluster.scale(jobs=1)

### Checking our slurm job in the queue 

In [10]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4719583   compute dask-wor  b382267 CG    4:18:08      1 l10106
           4721900   compute dask-wor  b382267  R       0:24      1 l20500
           4719490 interacti interact  b382267 CG    4:28:56      1 l40048
           4716625 interacti interact  b382267 CG    7:43:59      1 l40050


0

### Canceling the job if it is necessary. Set the job ID manually

In [9]:
Job_ID =   4719583
os.system("scancel " +str(Job_ID)) 

0

### Setting the path to the diagnostic repository 

In [11]:
sys.path.append(str(pwd)+'/../')
import src.shared_func
import src.tr_pr_mod

### Loading the extra function, which can be used in any diagnostic

In [12]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update,  data_size
        break
    except NameError and AttributeError:
        import src.shared_func
        from  src.shared_func import time_interpreter,   animation_creator, image_creator,  xarray_attribute_update,  data_size
        break

### Loading the tropical precipitation module

In [13]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError and AttributeError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [14]:
configdir = '../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'

### Setting the class attributes

In [15]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-6)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

### Importing the aqua module 

In [16]:
import aqua

In [17]:
from aqua import Reader
from aqua.reader import catalogue

In [18]:
catalogue(configdir=configdir)

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- lra-r100-day	
	- lra-r100-mon	
IFS	test-tco2559	4km experiment, coupled with FESOM
	- ICMGG_atm2d	2d output
	- ICMU_atm3d	3d output

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng

levante:
  args:
    path: ../../../config//levante/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# FESOM data

In [19]:
reader = Reader(model="FESOM", exp="tco2559-ng5", source="original_2d", configdir=configdir)
#FESOM = reader.retrieve(fix=False)
#FESOM = reader.retrieve(apply_unit_fix=False)
FESOM = reader.retrieve()

No fixes defined for model FESOM, experiment tco2559-ng5, source original_2d


In [28]:
FESOM

<xarray.Dataset>
Dimensions:   (time: 8328, nod2: 7402886)
Coordinates:
  * time      (time) datetime64[ns] 2020-01-20T00:56:00 ... 2020-12-31T23:56:00
Dimensions without coordinates: nod2
Data variables: (12/19)
    MLD1      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    MLD2      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    a_ice     (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmice_x  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmice_y  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    atmoce_x  (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    ...        ...
    ssh       (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    sss       (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    sst       (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    uice      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    vice      (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
    vve_5     (time, nod2) float32 dask.array<chunksize=(1, 7402886), meta=np.ndarray>
Attributes: (12/17)
    FESOM_model:                         FESOM2
    FESOM_website:                       fesom.de
    FESOM_git_SHA:                       469e74b1
    FESOM_MeshPath:                      /ec/res4/scratch/natr/RAPS/20/inputs...
    FESOM_mesh_representative_checksum:  
    FESOM_ClimateDataPath:               /ec/res4/scratch/natr/RAPS/20/inputs...
    ...                                  ...
    FESOM_use_floatice:                  0
    FESOM_whichEVP:                      1
    FESOM_evp_rheol_steps:               120
    FESOM_opt_visc:                      5
    FESOM_use_wsplit:                    -1
    history:                             2023-04-13 00:29:28 retrieved by AQU...

### size of entire dataset

In [23]:
data_size(FESOM)

61651234608

### timestep of dataset is

In [24]:
time_interpreter(FESOM)

'1H'

### Selecting the small part of dataset for tests

In [26]:
FESOM_small = FESOM['prec'][10:20,:]

### size of selected chunk

In [29]:
data_size(FESOM_small)

74028860

### Attribute update

In [31]:
#FESOM_small = xarray_attribute_update(FESOM_small, FESOM)

### Load selected chunk into a memory

In [32]:
FESOM_small = FESOM_small.compute()

In [ ]:
FESOM_small

### Units of precepitation rate

In [36]:
FESOM_small.attrs['units']

'm/s'

In [38]:
time_interpreter(FESOM_small) 

'1H'

### Name of coordinates

In [39]:
diag.coordinate_names(FESOM_small)

(None, None)

### Tropical latitudes

In [43]:
#FESOM_small = diag.ds_per_lat_range(FESOM_small)

In [ ]:
FESOM_small

### Fastest histogram

In [45]:
hist_fast_FESOM  = diag.hist1d_fast(FESOM_small,  preprocess = False)
hist_fast_FESOM

<xarray.DataArray (bin: 15)>
array([6.8875656e+07, 1.8191830e+06, 8.8119100e+05, 5.6015600e+05,
       3.8587300e+05, 2.7803500e+05, 2.0489300e+05, 1.5421600e+05,
       1.1936400e+05, 9.4678000e+04, 7.8318000e+04, 6.4756000e+04,
       5.4642000e+04, 4.5386000e+04, 3.8603000e+04])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-08 1.333e-07 ... 8e-07 8.667e-07 9.333e-07
Attributes:
    description:  precicipation rain
    long_name:    precicipation rain
    units:        m/s

### Plot of obtained histogram 

In [47]:
diag.hist_plot(hist_fast_FESOM, pdf = True, smooth = False, label='FESOM', plot_title = "FESOM, precipitation rate for one timestep")

## Plot of mean and median value of precipitation rate

## Add grid and time units!

In [51]:
diag.mean_and_median_plot(FESOM_small, savelabel = 'FESOM')

In [50]:
FESOM_small = FESOM['prec'][10:150,:]
FESOM_small = FESOM_small.compute()

## Check if vmin and vmax == None. Now its giving an error

In [ ]:
#animation_creator(MSWEP_small, vmin =0, vmax = 10**(-7),  trop_lat = 90,  label='MSWEP')

## Add units on bar

In [21]:
#image_creator(MSWEP_small, vmin =0, vmax = 10**(-7), trop_lat = 90, label='MSWEP')

Done!
